In [1]:
# Análise dos 15 Logradouros com Maior Média de Pessoas - VERSÃO SIMPLIFICADA V2
# Base: Planilha PROCESSADA
# Usa: APENAS coluna "Logradouro" (sem campos parseados)
# Período: Solicitado ao usuário
# Novidades: Top 15 Média (com médias por período) + Top 15 Qtd Aglo

# %% [markdown]
# # 1. Configuração Inicial

# %%
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliotecas importadas")
print(f"✓ Análise iniciada em: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")

# %% [markdown]
# # 2. Definir Período de Análise

# %%
print("\n" + "=" * 80)
print("DEFINIR PERÍODO DO RELATÓRIO")
print("=" * 80)

data_inicio_str = input("\nDigite a DATA INICIAL (dd/mm/aaaa): ")
data_fim_str = input("Digite a DATA FINAL (dd/mm/aaaa): ")

# Converter datas
try:
    data_inicio_dt = datetime.strptime(data_inicio_str, "%d/%m/%Y")
    data_fim_dt = datetime.strptime(data_fim_str, "%d/%m/%Y")
    
    # Converter para formato string usado nos filtros
    data_inicio = data_inicio_dt.strftime('%Y-%m-%d')
    data_fim = data_fim_dt.strftime('%Y-%m-%d')
    
    print(f"\n✓ Período selecionado: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    
    # Calcular quantidade de dias
    dias_periodo = (data_fim_dt - data_inicio_dt).days + 1
    total_periodos = dias_periodo * 4  # 4 períodos por dia
    
    print(f"\n{'=' * 80}")
    print(f"Período: {data_inicio_dt.strftime('%d/%m/%Y')} a {data_fim_dt.strftime('%d/%m/%Y')}")
    print(f"  • Dias: {dias_periodo}")
    print(f"  • Períodos por dia: 4")
    print(f"  • Total de períodos: {total_periodos}")
    print(f"{'=' * 80}")
    
except ValueError:
    print("❌ Formato de data inválido! Use dd/mm/aaaa")
    raise Exception("Formato de data inválido")

# %% [markdown]
# # 3. Selecionar Arquivo Processado

# %%
from pathlib import Path

print("\n" + "=" * 80)
print("SELEÇÃO DO ARQUIVO PROCESSADO")
print("=" * 80)

# Detectar raiz do projeto
script_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
if script_dir.name == 'notebooks':
    project_root = script_dir.parent
elif script_dir.name == 'etl':
    project_root = script_dir.parent.parent
else:
    project_root = script_dir

pasta_processed = project_root / 'data' / 'processed'
print(f"📂 Pasta processed: {pasta_processed}")

# Listar arquivos disponíveis
arquivos_disponiveis = sorted(list(pasta_processed.glob('*.xlsx')), 
                               key=lambda x: x.stat().st_mtime, 
                               reverse=True)

if arquivos_disponiveis:
    print(f"\n📁 Arquivos disponíveis (mais recentes primeiro):")
    for i, arq in enumerate(arquivos_disponiveis, 1):
        modificado = datetime.fromtimestamp(arq.stat().st_mtime).strftime('%d/%m/%Y %H:%M')
        print(f"  {i}. {arq.name}")
        print(f"     Modificado em: {modificado}\n")
    
    print("=" * 80)
    selecao = int(input("Digite o número do arquivo que deseja analisar: "))
    arquivo_selecionado = arquivos_disponiveis[selecao - 1]
    print(f"✓ Arquivo selecionado: {arquivo_selecionado.name}")
else:
    print(f"\n⚠️ Nenhum arquivo .xlsx encontrado em '{pasta_processed}'")
    print("   Execute primeiro: python src/etl/aplicar_parser_contagem_diaria.py")
    raise FileNotFoundError(f"Nenhum arquivo em {pasta_processed}")

print("=" * 80)

# %% [markdown]
# # 4. Carregar Dados Processados

# %%
print("\n" + "=" * 80)
print("CARREGANDO DADOS PROCESSADOS")
print("=" * 80)

try:
    df = pd.read_excel(arquivo_selecionado)
    print(f"✓ Arquivo carregado: {arquivo_selecionado.name}")
    print(f"✓ Total de registros: {len(df):,}")
    print(f"\nColunas disponíveis:")
    for col in df.columns:
        print(f"  - {col}")
    
    # Verificar campos necessários (APENAS os essenciais)
    campos_necessarios = ['Data', 'Logradouro', 'Período', 'Qtd. pessoas']
    campos_faltando = [c for c in campos_necessarios if c not in df.columns]
    
    if campos_faltando:
        print(f"\n✗ ERRO: Campos faltando: {campos_faltando}")
        raise KeyError("Campos necessários não encontrados")
    
except Exception as e:
    print(f"✗ ERRO ao carregar arquivo: {str(e)}")
    raise

# %% [markdown]
# # 5. Preparar Dados

# %%
print("\n" + "=" * 80)
print("PREPARANDO DADOS")
print("=" * 80)

# Converter data
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# Garantir que Qtd. pessoas seja numérico
df['Qtd. pessoas'] = pd.to_numeric(df['Qtd. pessoas'], errors='coerce').fillna(0)

# Limpar espaços em branco no Logradouro e Período
df['Logradouro'] = df['Logradouro'].astype(str).str.strip()
df['Período'] = df['Período'].astype(str).str.strip()

print(f"✓ Campo 'Data' convertido para datetime")
print(f"✓ Campo 'Qtd. pessoas' convertido para numérico")
print(f"✓ Campo 'Logradouro' limpo")
print(f"✓ Registros preparados: {len(df):,}")

# Verificar períodos únicos
print(f"\nPeríodos encontrados:")
for periodo in sorted(df['Período'].unique()):
    print(f"  - {periodo}")

# %% [markdown]
# # 6. Filtrar Período

# %%
print("\n" + "=" * 80)
print("FILTRANDO PERÍODO")
print("=" * 80)

# Filtrar período
df_periodo = df[
    (df['Data'] >= data_inicio) & 
    (df['Data'] <= data_fim)
].copy()

print(f"Data início: {data_inicio}")
print(f"Data fim: {data_fim}")
print(f"Dias no período: {dias_periodo}")
print(f"✓ Registros no período: {len(df_periodo):,}")
print(f"  Logradouros únicos: {df_periodo['Logradouro'].nunique()}")
print(f"  Períodos únicos: {sorted(df_periodo['Período'].unique())}")

# %% [markdown]
# # 7. Agrupar e Calcular Médias

# %%
print("\n" + "=" * 80)
print("AGRUPANDO E CALCULANDO MÉDIAS (POR LOGRADOURO)")
print("=" * 80)

# Agrupar APENAS por Logradouro (sem separar por período)
df_agrupado = df_periodo.groupby('Logradouro').agg({
    'Qtd. pessoas': 'sum'
}).reset_index()

# Renomear coluna
df_agrupado.rename(columns={'Qtd. pessoas': 'Soma pessoas'}, inplace=True)

# Calcular Média pessoas (soma / total de períodos)
df_agrupado['Média pessoas'] = df_agrupado['Soma pessoas'] / total_periodos

# Ordenar por Média pessoas (decrescente)
df_agrupado = df_agrupado.sort_values('Média pessoas', ascending=False)

print(f"✓ Agrupamento concluído")
print(f"  Total de logradouros: {len(df_agrupado)}")
print(f"  Maior Média pessoas: {df_agrupado['Média pessoas'].max():.2f}")
print(f"  Menor Média pessoas: {df_agrupado['Média pessoas'].min():.2f}")
print(f"\n  Cálculo da média: Soma pessoas ÷ {total_periodos} ({dias_periodo} dias × 4 períodos)")

# %% [markdown]
# # 8. Selecionar Top 15 Média

# %%
print("\n" + "=" * 80)
print("SELECIONANDO TOP 15 MÉDIA")
print("=" * 80)

# Pegar os 15 primeiros
df_top15 = df_agrupado.head(15).copy()

print(f"✓ Top 15 Média selecionados")
print("\nPRÉVIA DO TOP 15 MÉDIA:")
print("=" * 80)

for idx, row in df_top15.iterrows():
    posicao = list(df_top15.index).index(idx) + 1
    print(f"{posicao}. {row['Logradouro']}")
    print(f"   Soma pessoas: {row['Soma pessoas']:,.0f}")
    print(f"   Média pessoas (por período): {row['Média pessoas']:.2f}")
    print()

# %% [markdown]
# # 9. Calcular Médias por Período

# %%
print("\n" + "=" * 80)
print("CALCULANDO MÉDIAS POR PERÍODO")
print("=" * 80)

# Mapear os nomes dos períodos
mapeamento_periodos = {
    '05h - Madrugada': 'Madrugada',
    '10h - Manhã': 'Manhã',
    '15h - Tarde': 'Tarde',
    '20h - Noite': 'Noite'
}

# Padronizar nomes dos períodos
df_periodo['Período_padrao'] = df_periodo['Período'].map(mapeamento_periodos)

# Para cada logradouro do Top 15, calcular média por período
medias_por_periodo = []

for logradouro in df_top15['Logradouro']:
    df_log = df_periodo[df_periodo['Logradouro'] == logradouro].copy()
    
    # Agrupar por período e somar
    soma_por_periodo = df_log.groupby('Período_padrao')['Qtd. pessoas'].sum()
    
    # Calcular média (dividir pela quantidade de dias)
    media_madrugada = soma_por_periodo.get('Madrugada', 0) / dias_periodo
    media_manha = soma_por_periodo.get('Manhã', 0) / dias_periodo
    media_tarde = soma_por_periodo.get('Tarde', 0) / dias_periodo
    media_noite = soma_por_periodo.get('Noite', 0) / dias_periodo
    
    medias_por_periodo.append({
        'Logradouro': logradouro,
        'Média Madrugada': media_madrugada,
        'Média Manhã': media_manha,
        'Média Tarde': media_tarde,
        'Média Noite': media_noite
    })

df_medias_periodo = pd.DataFrame(medias_por_periodo)

# Merge com df_top15
df_top15_completo = df_top15.merge(df_medias_periodo, on='Logradouro')

print(f"✓ Médias por período calculadas para os Top 15")

# %% [markdown]
# # 10. Calcular Top 15 Aglomerações

# %%
print("\n" + "=" * 80)
print("CALCULANDO TOP 15 AGLOMERAÇÕES (>10 PESSOAS)")
print("=" * 80)

# Filtrar apenas registros com aglomeração (>10 pessoas)
df_aglomeracoes = df_periodo[df_periodo['Qtd. pessoas'] > 10].copy()

print(f"✓ Total de registros com aglomeração: {len(df_aglomeracoes):,}")

# Padronizar nomes dos períodos
df_aglomeracoes['Período_padrao'] = df_aglomeracoes['Período'].map(mapeamento_periodos)

# Contar aglomerações por logradouro e período
contagem_aglo = []

for logradouro in df_aglomeracoes['Logradouro'].unique():
    df_log = df_aglomeracoes[df_aglomeracoes['Logradouro'] == logradouro]
    
    # Contar por período
    qtd_madrugada = len(df_log[df_log['Período_padrao'] == 'Madrugada'])
    qtd_manha = len(df_log[df_log['Período_padrao'] == 'Manhã'])
    qtd_tarde = len(df_log[df_log['Período_padrao'] == 'Tarde'])
    qtd_noite = len(df_log[df_log['Período_padrao'] == 'Noite'])
    
    total_aglomeracoes = qtd_madrugada + qtd_manha + qtd_tarde + qtd_noite
    
    contagem_aglo.append({
        'Logradouro': logradouro,
        'Total Aglomerações': total_aglomeracoes,
        'Qtd Madrugada': qtd_madrugada,
        'Qtd Manhã': qtd_manha,
        'Qtd Tarde': qtd_tarde,
        'Qtd Noite': qtd_noite
    })

df_aglo = pd.DataFrame(contagem_aglo)

# Ordenar por Total Aglomerações (decrescente)
df_aglo = df_aglo.sort_values('Total Aglomerações', ascending=False)

# Selecionar Top 15
df_top15_aglo = df_aglo.head(15).copy()

print(f"✓ Top 15 Aglomerações calculado")
print(f"  Logradouros com aglomeração: {len(df_aglo)}")
print(f"  Maior quantidade: {df_top15_aglo['Total Aglomerações'].max()}")

print("\nPRÉVIA DO TOP 15 AGLOMERAÇÕES:")
print("=" * 80)

for idx, row in df_top15_aglo.iterrows():
    posicao = list(df_top15_aglo.index).index(idx) + 1
    print(f"{posicao}. {row['Logradouro']}")
    print(f"   Total: {row['Total Aglomerações']} aglomerações")
    print(f"   Madrugada: {row['Qtd Madrugada']} | Manhã: {row['Qtd Manhã']} | Tarde: {row['Qtd Tarde']} | Noite: {row['Qtd Noite']}")
    print()

# %% [markdown]
# # 11. Criar Abas no Excel

# %%
print("\n" + "=" * 80)
print("CRIANDO ABAS NO EXCEL")
print("=" * 80)

# Caminho do arquivo de saída
pasta_docs = project_root / 'docs'
pasta_docs.mkdir(parents=True, exist_ok=True)
output_file = pasta_docs / 'top15_media_diaria_pessoas.xlsx'

# Criar o arquivo Excel
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    
    # Aba 1: Top 15 Média (com médias por período)
    df_top15_media_exportar = df_top15_completo[[
        'Logradouro', 
        'Soma pessoas', 
        'Média pessoas',
        'Média Madrugada',
        'Média Manhã',
        'Média Tarde',
        'Média Noite'
    ]].copy()
    df_top15_media_exportar.insert(0, '#', range(1, len(df_top15_media_exportar) + 1))
    df_top15_media_exportar.to_excel(writer, sheet_name='Top 15 Média', index=False)
    print(f"✓ Aba 'Top 15 Média' criada")
    
    # Aba 2: Top 15 Qtd Aglo
    df_top15_aglo_exportar = df_top15_aglo.copy()
    df_top15_aglo_exportar.insert(0, '#', range(1, len(df_top15_aglo_exportar) + 1))
    df_top15_aglo_exportar.to_excel(writer, sheet_name='Top 15 Qtd Aglo', index=False)
    print(f"✓ Aba 'Top 15 Qtd Aglo' criada")
    
    # Criar uma aba para cada logradouro do Top 15 Média
    for idx, row in df_top15.iterrows():
        logradouro = row['Logradouro']
        soma_pessoas = row['Soma pessoas']
        media_pessoas = row['Média pessoas']
        
        # Filtrar dados originais para esse logradouro
        df_detalhado = df_periodo[
            df_periodo['Logradouro'] == logradouro
        ].copy()
        
        # Criar pivot: transformar períodos em colunas
        df_pivot = df_detalhado.pivot_table(
            index='Data',
            columns='Período',
            values='Qtd. pessoas',
            aggfunc='sum',
            fill_value=0
        ).reset_index()
        
        # Criar range completo de datas do período
        data_inicio_dt_range = pd.to_datetime(data_inicio)
        data_fim_dt_range = pd.to_datetime(data_fim)
        todas_datas = pd.date_range(start=data_inicio_dt_range, end=data_fim_dt_range, freq='D')
        
        # Criar DataFrame com todas as datas
        df_todas_datas = pd.DataFrame({'Data': todas_datas})
        
        # Fazer merge para incluir datas faltantes
        df_pivot = df_todas_datas.merge(df_pivot, on='Data', how='left')
        
        # Preencher valores faltantes com 0 (colunas de período)
        colunas_periodo = [col for col in df_pivot.columns if col != 'Data']
        df_pivot[colunas_periodo] = df_pivot[colunas_periodo].fillna(0)
        
        # Mapear os nomes dos períodos para os esperados
        mapeamento_periodos_export = {
            '05h - Madrugada': 'Madrugada',
            '10h - Manhã': 'Manhã',
            '15h - Tarde': 'Tarde',
            '20h - Noite': 'Noite'
        }
        
        # Renomear colunas de período
        df_pivot = df_pivot.rename(columns=mapeamento_periodos_export)
        
        # Garantir que todas as colunas de período existam
        for periodo in ['Madrugada', 'Manhã', 'Tarde', 'Noite']:
            if periodo not in df_pivot.columns:
                df_pivot[periodo] = 0
        
        # Preparar DataFrame para exportação na ordem correta
        df_exportar = pd.DataFrame()
        df_exportar['Logradouro'] = [logradouro] * len(df_pivot)
        df_exportar['Data'] = df_pivot['Data'].dt.strftime('%d/%m/%Y')
        df_exportar['Madrugada'] = df_pivot['Madrugada'].astype(int)
        df_exportar['Manhã'] = df_pivot['Manhã'].astype(int)
        df_exportar['Tarde'] = df_pivot['Tarde'].astype(int)
        df_exportar['Noite'] = df_pivot['Noite'].astype(int)
        
        # Nome da aba (limitado a 31 caracteres)
        posicao = list(df_top15.index).index(idx) + 1
        nome_aba = f"{posicao}. {logradouro[:25]}"
        if len(nome_aba) > 31:
            nome_aba = nome_aba[:31]
        
        # Escrever aba
        df_exportar.to_excel(writer, sheet_name=nome_aba, index=False)
        print(f"✓ Aba '{nome_aba}' criada ({len(df_exportar)} registros)")

print(f"\n✓ Arquivo criado: {output_file}")

# %% [markdown]
# # 12. Resumo Executivo

# %%
print("\n" + "=" * 80)
print("RESUMO EXECUTIVO")
print("=" * 80)

print(f"""
ANÁLISE: TOP 15 LOGRADOUROS (VERSÃO SIMPLIFICADA V2)

ARQUIVO ANALISADO:
• {arquivo_selecionado.name}

PERÍODO:
• De: {data_inicio_dt.strftime('%d/%m/%Y')}
• Até: {data_fim_dt.strftime('%d/%m/%Y')}
• Total de dias: {dias_periodo}
• Total de períodos: {total_periodos} ({dias_periodo} dias × 4 períodos)

METODOLOGIA:
• Fonte: Planilha PROCESSADA
• Campo usado: APENAS coluna "Logradouro" (sem campos parseados)
• Agrupamento: Por Logradouro (sem separar por período)

CÁLCULOS - TOP 15 MÉDIA:
• Soma pessoas: Soma de Qtd. pessoas de todos os períodos
• Média pessoas: Soma pessoas ÷ {total_periodos} períodos
• Média por período: Soma do período ÷ {dias_periodo} dias

CÁLCULOS - TOP 15 AGLOMERAÇÕES:
• Aglomeração: Qtd. pessoas > 10
• Contagem por período: Quantidade de ocorrências de aglomeração em cada período
• Total Aglomerações: Soma de todas as aglomerações (todos os períodos)

DADOS PROCESSADOS:
• Registros totais no período: {len(df_periodo):,}
• Logradouros únicos: {df_periodo['Logradouro'].nunique()}
• Registros com aglomeração (>10): {len(df_aglomeracoes):,}

ARQUIVO GERADO:
✓ {output_file}
  - Aba 'Top 15 Média': Resumo dos 15 com maior média (com médias por período)
  - Aba 'Top 15 Qtd Aglo': Resumo dos 15 com mais aglomerações (contagem por período)
  - 15 abas individuais: Detalhamento de cada logradouro do Top 15 Média

TOP 1 MÉDIA:
• Logradouro: {df_top15_completo.iloc[0]['Logradouro']}
• Soma pessoas: {df_top15_completo.iloc[0]['Soma pessoas']:,.0f}
• Média pessoas (por período): {df_top15_completo.iloc[0]['Média pessoas']:.2f}
• Média Madrugada: {df_top15_completo.iloc[0]['Média Madrugada']:.2f}
• Média Manhã: {df_top15_completo.iloc[0]['Média Manhã']:.2f}
• Média Tarde: {df_top15_completo.iloc[0]['Média Tarde']:.2f}
• Média Noite: {df_top15_completo.iloc[0]['Média Noite']:.2f}

TOP 1 AGLOMERAÇÕES:
• Logradouro: {df_top15_aglo.iloc[0]['Logradouro']}
• Total Aglomerações: {df_top15_aglo.iloc[0]['Total Aglomerações']}
• Qtd Madrugada: {df_top15_aglo.iloc[0]['Qtd Madrugada']}
• Qtd Manhã: {df_top15_aglo.iloc[0]['Qtd Manhã']}
• Qtd Tarde: {df_top15_aglo.iloc[0]['Qtd Tarde']}
• Qtd Noite: {df_top15_aglo.iloc[0]['Qtd Noite']}
""")

print("=" * 80)
print("✓ ANÁLISE CONCLUÍDA!")
print(f"✓ {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print("=" * 80)

✓ Bibliotecas importadas
✓ Análise iniciada em: 10/11/2025 12:13:52

DEFINIR PERÍODO DO RELATÓRIO

✓ Período selecionado: 01/10/2025 a 31/10/2025

Período: 01/10/2025 a 31/10/2025
  • Dias: 31
  • Períodos por dia: 4
  • Total de períodos: 124

SELEÇÃO DO ARQUIVO PROCESSADO
📂 Pasta processed: c:\Users\x504693\Documents\projetos\projeto_etl_dados\data\processed

📁 Arquivos disponíveis (mais recentes primeiro):
  1. Contagem_diaria_centro - Padronizada_processada_20251107_180111.xlsx
     Modificado em: 07/11/2025 18:01

  2. lista_logradouros_corretos.xlsx
     Modificado em: 04/11/2025 11:08

  3. cruzamento_nomes_base_okuhara.xlsx
     Modificado em: 21/10/2025 18:58

  4. cruzamento_nomes_base_tablet.xlsx
     Modificado em: 21/10/2025 18:51

✓ Arquivo selecionado: Contagem_diaria_centro - Padronizada_processada_20251107_180111.xlsx

CARREGANDO DADOS PROCESSADOS
✓ Arquivo carregado: Contagem_diaria_centro - Padronizada_processada_20251107_180111.xlsx
✓ Total de registros: 67,053

Col